1. Get a dictionary of the Processing Plants and the part created
2. Get a dictionary of the Products and the parts used.

In [5]:
import arcpy
import pandas as pd
import arcgis
from pprint import pprint
import random
import os

In [6]:
gdb = r"C:\DemoData\KNOW\SupplyChain\SupplyChain.gdb"
pplants = "ProcessingPlants"
products = "Products"
pplantPath = os.path.join(gdb, pplants)
productsPath = os.path.join(gdb, products)

In [7]:
sdf = pd.DataFrame.spatial.from_featureclass(pplantPath)
sdf.head(5)

,OBJECTID,X_WGS1984,Y_WGS1984,Type,Name,Material,Part,Distributors,PlantID,SHAPE
0,1,-117.125496,32.761458,Processing Plant,Plant 1,"Metals, Plastics",Part 1,"Distributor 2, Distributor 4, Distributor 9, D...",1,"{""x"": -117.12549565567934, ""y"": 32.76145804719..."
1,2,29.006001,41.066001,Processing Plant,Plant 2,"Aluminum, Plastics, Steel, Metals",Part 2,"Distributor 6, Distributor 10, Distributor 12,...",2,"{""x"": 29.00600140272212, ""y"": 41.0660009628414..."
2,3,72.826002,19.077003,Processing Plant,Plant 3,"Plastics, Steel, Metals",Part 3,"Distributor 6, Distributor 10, Distributor 12,...",3,"{""x"": 72.82600233462313, ""y"": 19.0770029830576..."
3,4,90.407142,23.709915,Processing Plant,Plant 4,"Steel, Plastics",Part 4,"Distributor 2, Distributor 4, Distributor 9, D...",4,"{""x"": 90.40714232325718, ""y"": 23.7099149438166..."
4,5,120.165003,30.252996,Processing Plant,Plant 5,"Plastics, Aluminum",Part 5,"Distributor 1, Distributor 3, Distributor 7, D...",5,"{""x"": 120.16500334735883, ""y"": 30.252996010603..."


In [22]:
partsFromPlant = {}
parts = list(sdf['Part'].unique())
for part in parts:
    pplant = list(sdf.loc[sdf['Part'] == part]['Name'])[0]
    partsFromPlant[pplant]=part
print(partsFromPlant)

Part 1
Part 2
Part 3
Part 4
Part 5
Part 6
Part 7
{'Plant 1': 'Part 1', 'Plant 2': 'Part 2', 'Plant 3': 'Part 3', 'Plant 4': 'Part 4', 'Plant 5': 'Part 5', 'Plant 6': 'Part 6', 'Plant 7': 'Part 7'}


In [ ]:
companyList = [""]
uCur = arcpy.da.UpdateCursor(productsPath,["ProductName","Parts","Company","MSRP"])
i=0
for row in uCur:
    row[0] = f"Product {i}"
    i+=1
    #row[1] = random.choice(dict[row[0]])
    #print(row)
    #uCur.updateRow(row)
del uCur

In [26]:
from faker import Faker

fake = Faker()

def generate_company_name():
    return fake.company()

# Generate 10 random company names
for _ in range(10):
    print(generate_company_name())


ModuleNotFoundError: No module named 'faker'

In [24]:
!pip install faker

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/ff/7c/087b2d78a49c40b5db2977818672e2ef6550426255ea7120a47c95302771/Faker-22.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 1.7/1.7 MB 7.9 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
